# 🚀 Fitness-AQA Vision Pipeline (YOLOv8-Pose Edition)

## ⚙️ INSTRUCTIONS:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run **Step 1** (Installation). It will **automatically restart** the runtime.
3. After the restart, start from **Step 2**.

---

## 📦 Step 1: Install Dependencies (AUTO-RESTART)

**⚠️ This cell will restart the runtime!** After it finishes (you see 'Session crashed'), ignore the error and move to Step 2.

In [ ]:
# 1. Install YOLO and dependencies
!pip install ultralytics scipy opencv-python matplotlib -q

# 2. Downgrade numpy to fix binary incompatibility
!pip install "numpy<2.0.0" --force-reinstall -q

print("\n✅ Installation complete! RESTARTING RUNTIME to apply changes...")

import os
os.kill(os.getpid(), 9)

## ✅ Step 2: Verify Installation

Run this after the auto-restart above.

In [ ]:
import torch
import numpy as np
from ultralytics import YOLO
import scipy

print(f"✅ YOLOv8 Version: {YOLO().version if hasattr(YOLO(), 'version') else 'Loaded'}")
print(f"✅ NumPy Version: {np.__version__}")
print(f"✅ GPU Available: {torch.cuda.is_available()}")
print("\n🎉 Ready to process!")

## 📤 Step 3: Upload Your Video

In [ ]:
from google.colab import files
import os

uploaded = files.upload()
if uploaded:
    video_path = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {video_path}")

## 🔧 Step 4: Define Processor

In [ ]:
import json
from scipy.signal import savgol_filter

class YOLOPoseExtractor:
    def __init__(self, model_variant='yolov8s-pose.pt', device='cuda'):
        self.model = YOLO(model_variant)
        self.device = device

    def smooth_signal(self, keypoints, window_length=5, polyorder=2):
        if len(keypoints) < window_length: return keypoints
        smoothed = np.zeros_like(keypoints)
        for i in range(keypoints.shape[1]):
            smoothed[:, i, 0] = savgol_filter(keypoints[:, i, 0], window_length, polyorder)
            smoothed[:, i, 1] = savgol_filter(keypoints[:, i, 1], window_length, polyorder)
        return smoothed

    def normalize_signal(self, keypoints):
        normalized = np.zeros_like(keypoints)
        for f in range(len(keypoints)):
            frame_kps = keypoints[f]
            mid_shoulder = (frame_kps[5] + frame_kps[6]) / 2
            mid_hip = (frame_kps[11] + frame_kps[12]) / 2
            torso_len = np.linalg.norm(mid_shoulder - mid_hip)
            scale = 1.0 if torso_len < 1e-3 else 1.0 / torso_len
            normalized[f] = (frame_kps - mid_hip) * scale
        return normalized

    def process_video(self, video_path):
        results = self.model(video_path, stream=True, device=self.device, verbose=False)
        raw_keypoints, scores = [], []
        
        for result in results:
            if result.keypoints is not None and len(result.keypoints.data) > 0:
                kp_data = result.keypoints.data[0].cpu().numpy()
                raw_keypoints.append(kp_data[:, :2])
                scores.append(result.keypoints.conf[0].cpu().numpy())
            else:
                raw_keypoints.append(np.zeros((17, 2)))
                scores.append(np.zeros(17))

        raw_keypoints = np.array(raw_keypoints)
        smoothed = self.smooth_signal(raw_keypoints)
        normalized = self.normalize_signal(smoothed)
        
        return {
            "video_id": os.path.basename(video_path),
            "frame_count": len(raw_keypoints),
            "raw_keypoints": raw_keypoints.tolist(),
            "smoothed_keypoints": smoothed.tolist(),
            "normalized_keypoints": normalized.tolist(),
            "scores": np.array(scores).tolist()
        }

print("✅ Processor logic loaded!")

## 🚀 Step 5: Run Extraction

In [ ]:
extractor = YOLOPoseExtractor()
result = extractor.process_video(video_path)
with open('pullup_analysis.json', 'w') as f:
    json.dump(result, f)
print(f"\n✅ Done! Saved to pullup_analysis.json")

## 💾 Step 6: Download

In [ ]:
from google.colab import files
files.download('pullup_analysis.json')
print("✅ Check your downloads folder!")